# Facility location assignment

Instructor: Frans de Ruiter

For assignment and data, see https://www.fransderuiter.com/JADS/

***

### Setup

In [ ]:
import numpy as np
import pandas as pd
from pyomo.environ import *
import matplotlib.pyplot as plt # optional for plotting
from matplotlib import cm # optional for plotting

### Read data
Here we load the distance table between the cities.

In [ ]:
data_path = "https://www.fransderuiter.com/JADS/Facilitylocation/FacilityLocation.xlsx"
# Create pandas table
distances = pd.read_excel(data_path, sheet_name=0, header=0, skiprows=2, index_col=1)

# Show table
distances.head()

Some data processing

In [ ]:
#remove 1st column ('unnamed', because there is a comment in first column it was added in the database)
distances = distances.drop(distances.columns[0],1)

# Show table with removed column
distances.head()

In [ ]:
## Show index names
cities = distances.columns
print(cities)

### Model parameters

In [ ]:
# Fixed cost for opening a DC (same for each city)
fixed_cost = 150000

# demand equals 100 for each city
demand_per_city = 100

# Supply capacity of each city is 4000
M = 4000

# Create pandas dataframe with supply
city_params_dict = {'supply': M*np.ones(cities.size, int), 
                    'demand': demand_per_city*np.ones(cities.size, int), 
                    'fixed cost': fixed_cost*np.ones(cities.size, int)}
city_params = pd.DataFrame(data=city_params_dict, index = cities)
city_params.head()

In [ ]:
# extract the fixed costs for a city (in this case "Athens") as follows:
i_1 = 'Athens'
city_params.loc[i_1,'fixed cost']

In [ ]:
# extract the fixed costs for a city (in this case "Athens" to "Amsterdam") as follows:
i_1 = 'Athens'
j_1 = 'Amsterdam'
distances[i_1][j_1]

### Model implementation

In [ ]:
# TODO
# write your optimization model in the cell here by replacing the XXXX in the cells

# Create model
m = ConcreteModel()

# Variables
m.locations = Var(cities, within=Binary)
m.shipments = Var(cities, cities, within = NonNegativeReals)

# Objective
m.value = Objective(
    expr=sum( XXXX for i in cities)
    + sum( XXXX for i in cities for j in cities)
    , sense=minimize)
    
# Constraints on supplying only when facility is open
m.supply_restriction = ConstraintList()
for i in cities:
    m.supply_restriction.add( XXXX )

# Constraints on demand fulfillment
m.demand_fulfillment = ConstraintList()
for j in cities:
    m.demand_fulfillment.add( XXXX )

### Solve the model

In [ ]:
# Optimize
solver = SolverFactory('cbc') # Take the cbc solver, glpk is very slow
status = solver.solve(m,tee=False,) # Set tee to True to see log of the solver

# Print the status of the solved mixed integer linear model once it is done
print("Status = %s \n" % status.solver.termination_condition)

### Show the solution


In [ ]:
# Show some visualization via tables and/or plots to present your solution.
#TODO (optional): add more visualization, print your solution etc

# Make a vector with all the shipments
totalshipments = [sum(value(m.shipments[i,j]) for j in cities) for i in cities]

plt.figure(figsize=(3,10))

# make a horizontal bar plots
plt.barh(range(cities.size),totalshipments, color="blue",align="center") 

# Use city names on the vertical axis
plt.yticks(range(cities.size),cities,rotation=0)

# Set title and show plot
plt.title('Total shipments per DC',fontsize=20)
plt.show()